In [1]:
import numpy as np

In [2]:
WK1 = np.array([[1, 0, 1], [0, 1, 0], [1, 0, 1], [0, 1, 0]])
WV1 = np.array([[0, 1, 1], [1, 0, 0], [1, 0, 1], [0, 1, 0]])
WQ1 = np.array([[0, 0, 0], [1, 1, 0], [0, 0, 1], [1, 0, 0]])

WK2 = np.array([[0, 1, 1], [1, 0, 1], [1, 1, 0], [0, 1, 0]])
WV2 = np.array([[1, 0, 0], [0, 1, 1], [0, 0, 1], [1, 0, 0]])
WQ2 = np.array([[1, 0, 1], [0, 1, 0], [1, 0, 0], [0, 1, 1]])

In [3]:
embedding = np.array([[1, 3, 3, 5], [2.84, 3.99, 4, 6]])
K1 = embedding @ WK1
K1

array([[4.  , 8.  , 4.  ],
       [6.84, 9.99, 6.84]])

In [4]:
V1 = embedding @ WV1
V1

array([[6.  , 6.  , 4.  ],
       [7.99, 8.84, 6.84]])

In [5]:
Q1 = embedding @ WQ1
Q1

array([[8.  , 3.  , 3.  ],
       [9.99, 3.99, 4.  ]])

In [6]:
scores1 = Q1 @ K1.T
scores1

array([[ 68.    , 105.21  ],
       [ 87.88  , 135.5517]])

In [7]:
scores1 = scores1 / np.sqrt(3)
scores1

array([[39.2598183 , 60.74302182],
       [50.73754166, 78.26081048]])

In [8]:
def softmax(x):
    x = np.array(x)  # Convert input to numpy array
    if len(x.shape) == 1:  # Handle 1D arrays by adding a new axis
        x = x[np.newaxis, :]
    e_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return e_x / np.sum(e_x, axis=1, keepdims=True)


scores1 = softmax(scores1)
scores1

array([[4.67695573e-10, 1.00000000e+00],
       [1.11377182e-12, 1.00000000e+00]])

In [9]:
attention1 = scores1 @ V1
attention1

array([[7.99, 8.84, 6.84],
       [7.99, 8.84, 6.84]])

In [10]:
def attention(x, WQ, WK, WV):
    K = x @ WK
    V = x @ WV
    Q = x @ WQ

    scores = Q @ K.T
    scores = scores / np.sqrt(3)
    scores = softmax(scores)
    scores = scores @ V
    return scores

In [11]:
attention1 = attention(embedding, WQ1, WK1, WV1)
attention1

array([[7.99, 8.84, 6.84],
       [7.99, 8.84, 6.84]])

In [12]:
attention2 = attention(embedding, WQ2, WK2, WV2)
attention2

array([[8.84, 3.99, 7.99],
       [8.84, 3.99, 7.99]])

In [13]:
attentions = np.concatenate([attention1, attention2], axis=1)
attentions

array([[7.99, 8.84, 6.84, 8.84, 3.99, 7.99],
       [7.99, 8.84, 6.84, 8.84, 3.99, 7.99]])

In [14]:
# Just some random values
W = np.array(
    [
        [0.79445237, 0.1081456, 0.27411536, 0.78394531],
        [0.29081936, -0.36187258, -0.32312791, -0.48530339],
        [-0.36702934, -0.76471963, -0.88058366, -1.73713022],
        [-0.02305587, -0.64315981, -0.68306653, -1.25393866],
        [0.29077448, -0.04121674, 0.01509932, 0.13149906],
        [0.57451867, -0.08895355, 0.02190485, 0.24535932],
    ]
)
Z = attentions @ W
Z

array([[ 11.95481735, -14.12627891, -12.49250332, -18.50804518],
       [ 11.95481735, -14.12627891, -12.49250332, -18.50804518]])

In [15]:
W1 = np.random.randn(4, 8)
W2 = np.random.randn(8, 4)
b1 = np.random.randn(8)
b2 = np.random.randn(4)

In [16]:
def relu(x):
    return np.maximum(0, x)

def feed_forward(Z, W1, b1, W2, b2):
    return relu(Z.dot(W1) + b1).dot(W2) + b2

In [17]:
output_encoder = feed_forward(Z, W1, b1, W2, b2)
output_encoder

array([[ 39.23832613,  10.94504339, -10.11288266, -31.57712331],
       [ 39.23832613,  10.94504339, -10.11288266, -31.57712332]])

In [18]:
d_embedding = 4
d_key = d_value = d_query = 3
d_feed_forward = 8
n_attention_heads = 2

def attention(x, WQ, WK, WV):
    K = x @ WK
    V = x @ WV
    Q = x @ WQ

    scores = Q @ K.T
    scores = scores / np.sqrt(d_key)
    scores = softmax(scores)
    scores = scores @ V
    return scores

def multi_head_attention(x, WQs, WKs, WVs):
    attentions = np.concatenate(
        [attention(x, WQ, WK, WV) for WQ, WK, WV in zip(WQs, WKs, WVs)], axis=1
    )
    W = np.random.randn(n_attention_heads * d_value, d_embedding)
    return attentions @ W

def feed_forward(Z, W1, b1, W2, b2):
    return relu(Z.dot(W1) + b1).dot(W2) + b2

def encoder_block(x, WQs, WKs, WVs, W1, b1, W2, b2):
    Z = multi_head_attention(x, WQs, WKs, WVs)
    Z = feed_forward(Z, W1, b1, W2, b2)
    return Z

def random_encoder_block(x):
    WQs = [
        np.random.randn(d_embedding, d_query) for _ in range(n_attention_heads)
    ]
    WKs = [
        np.random.randn(d_embedding, d_key) for _ in range(n_attention_heads)
    ]
    WVs = [
        np.random.randn(d_embedding, d_value) for _ in range(n_attention_heads)
    ]
    W1 = np.random.randn(d_embedding, d_feed_forward)
    b1 = np.random.randn(d_feed_forward)
    W2 = np.random.randn(d_feed_forward, d_embedding)
    b2 = np.random.randn(d_embedding)
    return encoder_block(x, WQs, WKs, WVs, W1, b1, W2, b2)

In [19]:
embedding

array([[1.  , 3.  , 3.  , 5.  ],
       [2.84, 3.99, 4.  , 6.  ]])

In [20]:
random_encoder_block(embedding)

array([[ 6.30544367, 55.35720351, -8.94552101, 36.31134929],
       [ 6.30144709, 55.30167828, -8.92573111, 36.25953458]])

In [21]:
def encoder(x, n=6):
    for _ in range(n):
        x = random_encoder_block(x)
    return x


encoder(embedding)

array([[40352854.00023742, 36941199.2395912 , -4769993.4050151 ,
        -3355840.30156874],
       [40352854.00023742, 36941199.2395912 , -4769993.4050151 ,
        -3355840.30156874]])

In [22]:
(embedding + Z).mean(axis=-1, keepdims=True)

array([[-5.29300251],
       [-4.08550252]])

In [23]:
(embedding + Z).std(axis=-1, keepdims=True)

array([[10.63170702],
       [10.99362604]])

In [24]:
def layer_norm(x, epsilon=1e-6):
    mean = x.mean(axis=-1, keepdims=True)
    std = x.std(axis=-1, keepdims=True)
    return (x - mean) / (std + epsilon)

def encoder_block(x, WQs, WKs, WVs, W1, b1, W2, b2):
    Z = multi_head_attention(x, WQs, WKs, WVs)
    Z = layer_norm(Z + x)

    output = feed_forward(Z, W1, b1, W2, b2)
    return layer_norm(output + Z)

In [25]:
layer_norm(Z + embedding)

array([[ 1.71635826, -0.54866785, -0.39499776, -0.77269265],
       [ 1.7173877 , -0.55038945, -0.40086868, -0.76612956]])

In [26]:
def encoder(x, n=6):
    for _ in range(n):
        x = random_encoder_block(x)
    return x


encoder(embedding)

array([[ 1.54019984, -0.65334771,  0.19205135, -1.07890348],
       [ 1.54019984, -0.65334771,  0.19205135, -1.07890348]])

In [27]:
d_embedding = 4
n_attention_heads = 2

E = np.array([[1, 1, 0, 1]])
WQs = [np.random.randn(d_embedding, d_query) for _ in range(n_attention_heads)]
WKs = [np.random.randn(d_embedding, d_key) for _ in range(n_attention_heads)]
WVs = [np.random.randn(d_embedding, d_value) for _ in range(n_attention_heads)]

Z_self_attention = multi_head_attention(E, WQs, WKs, WVs)
Z_self_attention

array([[-5.68583465,  8.65844136, -3.28747964, -0.78329174]])

In [28]:
Z_self_attention = layer_norm(Z_self_attention + E)
Z_self_attention

array([[-0.92256896,  1.64143633, -0.67261632, -0.04625105]])

In [29]:
def encoder_decoder_attention(encoder_output, attention_input, WQ, WK, WV):
    # The next three lines are the key difference!
    K = encoder_output @ WK    # Note that now we pass the previous encoder output!
    V = encoder_output @ WV    # Note that now we pass the previous encoder output!
    Q = attention_input @ WQ   # Same as self-attention

    # This stays the same
    scores = Q @ K.T
    scores = scores / np.sqrt(d_key)
    scores = softmax(scores)
    scores = scores @ V
    return scores


def multi_head_encoder_decoder_attention(
    encoder_output, attention_input, WQs, WKs, WVs
):
    # Note that now we pass the previous encoder output!
    attentions = np.concatenate(
        [
            encoder_decoder_attention(
                encoder_output, attention_input, WQ, WK, WV
            )
            for WQ, WK, WV in zip(WQs, WKs, WVs)
        ],
        axis=1,
    )
    W = np.random.randn(n_attention_heads * d_value, d_embedding)
    return attentions @ W

In [30]:
WQs = [np.random.randn(d_embedding, d_query) for _ in range(n_attention_heads)]
WKs = [np.random.randn(d_embedding, d_key) for _ in range(n_attention_heads)]
WVs = [np.random.randn(d_embedding, d_value) for _ in range(n_attention_heads)]

encoder_output = np.array([[-1.5, 1.0, -0.8, 1.5], [1.0, -1.0, -0.5, 1.0]])

Z_encoder_decoder = multi_head_encoder_decoder_attention(
    encoder_output, Z_self_attention, WQs, WKs, WVs
)
Z_encoder_decoder

array([[ 3.00573814, 12.60121576, -9.62652818, -5.07636579]])

In [31]:
Z_encoder_decoder = layer_norm(Z_encoder_decoder + Z_self_attention)
Z_encoder_decoder

array([[ 0.20164373,  1.5218806 , -1.14278738, -0.58073694]])

In [32]:
W1 = np.random.randn(4, 8)
W2 = np.random.randn(8, 4)
b1 = np.random.randn(8)
b2 = np.random.randn(4)

output = layer_norm(feed_forward(Z_encoder_decoder, W1, b1, W2, b2) + Z_encoder_decoder)
output

array([[-0.57909149,  1.71950627, -0.74002355, -0.40039123]])

In [33]:
d_embedding = 4
d_key = d_value = d_query = 3
d_feed_forward = 8
n_attention_heads = 2
encoder_output = np.array([[-1.5, 1.0, -0.8, 1.5], [1.0, -1.0, -0.5, 1.0]])

def decoder_block(
    x,
    encoder_output,
    WQs_self_attention, WKs_self_attention, WVs_self_attention,
    WQs_ed_attention, WKs_ed_attention, WVs_ed_attention,
    W1, b1, W2, b2,
):
    # Same as before
    Z = multi_head_attention(
        x, WQs_self_attention, WKs_self_attention, WVs_self_attention
    )
    Z = layer_norm(Z + x)

    # The next three lines are the key difference!
    Z_encoder_decoder = multi_head_encoder_decoder_attention(
        encoder_output, Z, WQs_ed_attention, WKs_ed_attention, WVs_ed_attention
    )
    Z_encoder_decoder = layer_norm(Z_encoder_decoder + Z)

    # Same as before
    output = feed_forward(Z_encoder_decoder, W1, b1, W2, b2)
    return layer_norm(output + Z_encoder_decoder)

def random_decoder_block(x, encoder_output):
    # Just a bunch of random initializations
    WQs_self_attention = [
        np.random.randn(d_embedding, d_query) for _ in range(n_attention_heads)
    ]
    WKs_self_attention = [
        np.random.randn(d_embedding, d_key) for _ in range(n_attention_heads)
    ]
    WVs_self_attention = [
        np.random.randn(d_embedding, d_value) for _ in range(n_attention_heads)
    ]

    WQs_ed_attention = [
        np.random.randn(d_embedding, d_query) for _ in range(n_attention_heads)
    ]
    WKs_ed_attention = [
        np.random.randn(d_embedding, d_key) for _ in range(n_attention_heads)
    ]
    WVs_ed_attention = [
        np.random.randn(d_embedding, d_value) for _ in range(n_attention_heads)
    ]

    W1 = np.random.randn(d_embedding, d_feed_forward)
    b1 = np.random.randn(d_feed_forward)
    W2 = np.random.randn(d_feed_forward, d_embedding)
    b2 = np.random.randn(d_embedding)


    return decoder_block(
        x, encoder_output,
        WQs_self_attention, WKs_self_attention, WVs_self_attention,
        WQs_ed_attention, WKs_ed_attention, WVs_ed_attention,
        W1, b1, W2, b2,
    )

In [34]:
def decoder(x, decoder_embedding, n=6):
    for _ in range(n):
        x = random_decoder_block(x, decoder_embedding)
    return x

decoder(E, encoder_output)

array([[-1.38703139, -0.34892723,  0.39489996,  1.34105865]])

In [35]:
def linear(x, W, b):
    return np.dot(x, W) + b

x = linear([1, 0, 1, 0], np.random.randn(4, 10), np.random.randn(10))
x

array([-0.67179275,  0.17198718, -1.69655828,  1.33603561,  0.86000803,
        2.43942626, -0.30797465,  0.77796867,  1.541172  ,  1.50069719])

In [36]:
softmax(x)

array([[0.01617342, 0.03760545, 0.00580433, 0.1204455 , 0.07482633,
        0.36306722, 0.02327051, 0.06893268, 0.14787003, 0.14200453]])

In [37]:
vocabulary = [
    "hello",
    "mundo",
    "world",
    "how",
    "?",
    "EOS",
    "SOS",
    "a",
    "hola",
    "c",
]
embedding_reps = np.random.randn(10, 4)
vocabulary_embeddings = {
    word: embedding_reps[i] for i, word in enumerate(vocabulary)
}
vocabulary_embeddings

{'hello': array([-1.06998789, -0.38733348, -0.30939307, -0.45660257]),
 'mundo': array([-0.76956495,  1.94292748,  1.05118722, -0.35117607]),
 'world': array([-1.0570816 , -0.00895372, -0.0984772 ,  0.08861666]),
 'how': array([ 0.08524699, -0.48298981,  0.26719934,  0.08619517]),
 '?': array([-0.05613469,  0.25952293, -0.14341733,  0.19177927]),
 'EOS': array([ 0.24207383, -0.5176897 , -0.53633206, -0.6855262 ]),
 'SOS': array([-0.39902152, -1.95851238, -0.64138367,  0.92162645]),
 'a': array([ 0.60303594, -0.48169978,  0.50561151, -0.53957713]),
 'hola': array([ 0.68195484, -1.90854728, -0.57311108, -0.16569104]),
 'c': array([-2.18418514,  1.78115469, -0.90508235, -1.39450771])}

In [38]:
def generate(input_sequence, max_iters=3):
    # We first encode the inputs into embeddings
    # This skips the positional encoding step for simplicity
    embedded_inputs = [
        vocabulary_embeddings[token] for token in input_sequence
    ]
    print("Embedding representation (encoder input)", embedded_inputs)

    # We then generate an embedding representation
    encoder_output = encoder(embedded_inputs)
    print("Embedding generated by encoder (encoder output)", encoder_output)

    # We initialize the decoder output with the embedding of the start token
    sequence_embeddings = [vocabulary_embeddings["SOS"]]
    output = "SOS"
    
    # Random matrices for the linear layer
    W_linear = np.random.randn(d_embedding, len(vocabulary))
    b_linear = np.random.randn(len(vocabulary))

    # We limit number of decoding steps to avoid too long sequences without EOS
    for i in range(max_iters):
        # Decoder step
        decoder_output = decoder(sequence_embeddings, encoder_output)

        # Only use the last output for prediction
        logits = linear(decoder_output[-1], W_linear, b_linear)
        # We wrap logits in a list as our softmax expects batches/2D array
        probs = softmax([logits])

        # We get the most likely next token
        next_token = vocabulary[np.argmax(probs)]
        sequence_embeddings.append(vocabulary_embeddings[next_token])
        output += " " + next_token

        print(
            "Iteration", i, 
            "next token", next_token,
            "with probability of", np.max(probs),
        )

        # If the next token is the end token, we return the sequence
        if next_token == "EOS":
            return output

    return output, sequence_embeddings

In [39]:
generate(["hello", "world"])

Embedding representation (encoder input) [array([-1.06998789, -0.38733348, -0.30939307, -0.45660257]), array([-1.0570816 , -0.00895372, -0.0984772 ,  0.08861666])]
Embedding generated by encoder (encoder output) [[ 1.39061441  0.49642812 -1.08367269 -0.80336984]
 [ 1.3906145   0.49642797 -1.08367269 -0.80336978]]
Iteration 0 next token how with probability of 0.4213570599466652
Iteration 1 next token how with probability of 0.8761176187808272
Iteration 2 next token SOS with probability of 0.4491314403338158


('SOS how how SOS',
 [array([-0.39902152, -1.95851238, -0.64138367,  0.92162645]),
  array([ 0.08524699, -0.48298981,  0.26719934,  0.08619517]),
  array([ 0.08524699, -0.48298981,  0.26719934,  0.08619517]),
  array([-0.39902152, -1.95851238, -0.64138367,  0.92162645])])